# Exploratory Data Analysis (EDA)

在本 Notebook 中，我們將示範如何對資料進行探索性資料分析 (EDA)，包括：
1. 基本統計量（平均值、中位數、標準差、分位數）
2. 相關性分析（數值特徵的 Pearson correlation，以及類別特徵的 Cramér's V）
3. 繪圖（箱型圖、直方圖、散佈圖、KDE、Q-Q 圖）
4. 資料分佈分析與群組比較
5. 使用 PCA 進行初步降維探索

我們將使用 sklearn 的 `iris` 資料集來做示範。該資料集包含 150 筆鳶尾花樣本，具有4個數值特徵（花萼長度、花萼寬度、花瓣長度、花瓣寬度）與1個類別特徵（花種）。

In [ ]:
import numpy as npimport pandas as pdimport matplotlib.pyplot as pltimport seaborn as snsfrom sklearn.datasets import load_irisfrom sklearn.decomposition import PCAfrom sklearn.preprocessing import StandardScalerfrom scipy import stats%matplotlib inlinesns.set(style="whitegrid", font_scale=1.2)# 載入 iris 資料集iris = load_iris()df = pd.DataFrame(data=np.c_[iris['data'], iris['target']], columns=iris['feature_names'] + ['target'])# 將 target 數值轉為類別名稱df['target'] = df['target'].map({0: iris['target_names'][0], 1: iris['target_names'][1], 2: iris['target_names'][2]})df.head()

## 1. 基本統計量 (Descriptive Statistics)
使用 `df.info()` 與 `df.describe()` 檢視基本資訊與統計量。

In [ ]:
df.info()

In [ ]:
df.describe()

查看類別特徵分佈：

In [ ]:
df['target'].value_counts()

## 2. 相關性分析 (Correlation Analysis)

### 數值特徵的相關性
透過 `df.corr()` 計算數值特徵的 Pearson 相關係數，並使用熱力圖顯示。

In [ ]:
corr_matrix = df.iloc[:,:4].corr()  # 前四欄為數值特徵plt.figure(figsize=(6,4))sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)plt.title('Correlation Matrix of Numerical Features')plt.show()

### 類別特徵的相關性 (Cramér's V)

雖然 iris 資料集僅有一個類別特徵 (`target`)，為示範計算 Cramér's V，我們將創建一個假的類別特徵 `pretend_cat`。

In [ ]:
from scipy.stats import chi2_contingencydef cramers_v(x, y):    confusion_matrix = pd.crosstab(x, y)    chi2 = chi2_contingency(confusion_matrix)[0]    n = confusion_matrix.sum().sum()    r,k = confusion_matrix.shape    return np.sqrt((chi2/(n*(min(k,r)-1))))df['pretend_cat'] = np.random.choice(['X','Y','Z'], size=len(df))cramers_v_value = cramers_v(df['target'], df['pretend_cat'])print(f"Cramér's V (target vs pretend_cat): {cramers_v_value:.3f}")

## 3. 繪圖 (Visualization)

### 箱型圖 (Boxplot)
檢視數值特徵的異常值與分佈情況。

In [ ]:
for col in iris['feature_names']:    plt.figure(figsize=(6,4))    sns.boxplot(y=df[col])    plt.title(f'Boxplot of {col}')    plt.show()

### 直方圖 (Histogram) & KDE
檢視特徵分佈形狀與密度。

In [ ]:
for col in iris['feature_names']:    plt.figure(figsize=(6,4))    sns.histplot(df[col], kde=True)    plt.title(f'Histogram & KDE of {col}')    plt.show()

### Q-Q 圖 (Q-Q Plot)
檢測特徵分佈與常態分佈之差異。

In [ ]:
import statsmodels.api as smfor col in iris['feature_names']:    plt.figure(figsize=(6,4))    sm.qqplot(df[col], line='s')    plt.title(f'Q-Q Plot of {col}')    plt.show()

### 散佈圖 (Scatter Plot) & Pairplot
觀察特徵間的兩兩關係與分佈。

In [ ]:
sns.pairplot(df, hue='target', diag_kind='kde')plt.show()

## 4. 資料分佈與群組比較

透過群組分析，如比較不同類別下的數值特徵平均值或中位數。

In [ ]:
print(df.groupby('target')['sepal length (cm)'].mean())# 使用箱型圖比較不同群組的數值分佈plt.figure(figsize=(8,4))sns.boxplot(x='target', y='sepal width (cm)', data=df)plt.title('Sepal Width by Target')plt.show()

## 5. 主成分分析 (PCA)

PCA 可將高維特徵降至低維便於視覺化並探索資料結構。

In [ ]:
X = df.iloc[:,:4]X_scaled = StandardScaler().fit_transform(X)pca = PCA(n_components=2)X_pca = pca.fit_transform(X_scaled)pca_df = pd.DataFrame(X_pca, columns=['PC1','PC2'])pca_df['target'] = df['target']plt.figure(figsize=(8,6))sns.scatterplot(x='PC1', y='PC2', hue='target', data=pca_df)plt.title('PCA Result')plt.show()print('Explained Variance Ratio:', pca.explained_variance_ratio_)print('Cumulative Variance Ratio:', np.cumsum(pca.explained_variance_ratio_))

透過 PCA 的前兩個主成分，我們可將高維度的特徵空間以 2D 圖表呈現，觀察資料點在低維空間的分佈情形，以及各主成分對資料變異解釋程度。

## 總結

透過 EDA 我們可了解資料的基本結構、特徵分佈、群組差異與特徵間關係，並使用 PCA 進行初步降維探索，為後續的特徵工程與模型訓練提供指引。